<a href="https://colab.research.google.com/github/AnkanMoh/MachineLearning/blob/main/T20WorldCup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
import pandas as pd

# Define the DataFrame
data = {
    'Team': ['Canada', 'India', 'Ireland', 'Pakistan', 'United States',
             'Australia', 'England', 'Namibia', 'Oman', 'Scotland',
             'Afghanistan', 'New Zealand', 'Papua New Guinea', 'Uganda', 'West Indies',
             'Bangladesh', 'Netherlands', 'South Africa', 'Sri Lanka', 'Nepal'],
    'Mat': [60, 221, 168, 243, 30,
            190, 186, 66, 82, 98,
            132, 217, 63, 94, 197,
            170, 109, 178, 191, 86],
    'Won': [31, 142, 71, 140, 17,
            102, 96, 42, 40, 46,
            81, 109, 35, 70, 85,
            66, 54, 98, 85, 49],
    'Lost': [26, 68, 88, 92, 10,
             81, 81, 22, 39, 47,
             48, 91, 27, 21, 99,
             100, 49, 76, 100, 34]
}

df = pd.DataFrame(data)
print(df)

                Team  Mat  Won  Lost
0             Canada   60   31    26
1              India  221  142    68
2            Ireland  168   71    88
3           Pakistan  243  140    92
4      United States   30   17    10
5          Australia  190  102    81
6            England  186   96    81
7            Namibia   66   42    22
8               Oman   82   40    39
9           Scotland   98   46    47
10       Afghanistan  132   81    48
11       New Zealand  217  109    91
12  Papua New Guinea   63   35    27
13            Uganda   94   70    21
14       West Indies  197   85    99
15        Bangladesh  170   66   100
16       Netherlands  109   54    49
17      South Africa  178   98    76
18         Sri Lanka  191   85   100
19             Nepal   86   49    34


In [27]:
df['Weighted_Average'] = (df['Mat'] * df['Won']) / (df['Mat'] + df['Lost'])

In [28]:
# Define groups
groups = {
    'Group A': ['Canada', 'India', 'Ireland', 'Pakistan', 'United States'],
    'Group B': ['Australia', 'England', 'Namibia', 'Oman', 'Scotland'],
    'Group C': ['Afghanistan', 'New Zealand', 'Papua New Guinea', 'Uganda', 'West Indies'],
    'Group D': ['Bangladesh', 'Netherlands', 'South Africa', 'Sri Lanka', 'Nepal']
}

print("\nGroups:")
for group, teams in groups.items():
    print(group + ":", ", ".join(teams))

def predict_match_winner(team1, team2, df):
    weighted_avg_team1 = df.loc[df['Team'] == team1, 'Weighted_Average'].values[0]
    weighted_avg_team2 = df.loc[df['Team'] == team2, 'Weighted_Average'].values[0]
    if weighted_avg_team1 > weighted_avg_team2:
        return team1
    elif weighted_avg_team2 > weighted_avg_team1:
        return team2
    else:
        return "It's a draw"

def predict_group_winners(groups, df):
    group_winners = {}
    match_no = 1
    for group, teams in groups.items():
        group_winners[group] = []
        print(f"\n{group}:")
        for i in range(len(teams)):
            for j in range(i+1, len(teams)):
                team1 = teams[i]
                team2 = teams[j]
                winner = predict_match_winner(team1, team2, df)
                group_winners[group].append(winner)
                print(f"Match {match_no}: {team1} vs {team2} -> Winner: {winner}")
                match_no += 1
    return group_winners

def create_super_8_table(group_winners):
    super_8 = pd.DataFrame(columns=['Group', 'Team'])
    for group, winners in group_winners.items():
        top_2 = pd.Series(winners).value_counts().nlargest(2).index
        super_8 = pd.concat([super_8, pd.DataFrame({'Group': [group]*2, 'Team': top_2})], ignore_index=True)
    return super_8

group_winners = predict_group_winners(groups, df)

for group_name, teams in groups.items():
    print(f"{group_name}: {', '.join(teams)}")

super_8_table = create_super_8_table(group_winners)

def divide_super_8(super_8_table):
    new_group_1 = []
    new_group_2 = []
    group_teams = {group: [] for group in groups.keys()}
    for idx, row in super_8_table.iterrows():
        group_teams[row['Group']].append(row['Team'])
    for group, teams in group_teams.items():
        if len(new_group_1) <= len(new_group_2):
            new_group_1.append(teams[0])
            new_group_2.append(teams[1])
        else:
            new_group_1.append(teams[1])
            new_group_2.append(teams[0])
    return pd.DataFrame({'Group 1': new_group_1, 'Group 2': new_group_2})

new_groups = divide_super_8(super_8_table)

print("\nSuper 8 Table:")
print(super_8_table.to_string(index=False))

print("\nGroups for Super 8:")
print(new_groups.to_string(index=False))

def simulate_matches(new_groups, df):
    results = {'Group 1': [], 'Group 2': []}
    match_no = 1
    for group in ['Group 1', 'Group 2']:
        print(f"\n{group}:")
        teams = new_groups[group].dropna().values
        for i in range(len(teams)):
            for j in range(i+1, len(teams)):
                team1 = teams[i]
                team2 = teams[j]
                winner = predict_match_winner(team1, team2, df)
                results[group].append((team1, team2, winner))
                print(f"Match {match_no}: {team1} vs {team2} -> Winner: {winner}")
                match_no += 1
    return results

match_results = simulate_matches(new_groups, df)

def determine_semi_finalists(match_results):
    semi_finalists = {'Group 1': [], 'Group 2': []}
    for group, matches in match_results.items():
        win_counts = {}
        for match in matches:
            winner = match[2]
            if winner != "It's a draw":
                if winner not in win_counts:
                    win_counts[winner] = 0
                win_counts[winner] += 1
        sorted_teams = sorted(win_counts, key=win_counts.get, reverse=True)
        semi_finalists[group] = sorted_teams[:2]
    return semi_finalists

semi_finalists = determine_semi_finalists(match_results)

def simulate_semi_finals(semi_finalists, df):
    print("\nSemi-Finals:")
    semi_final_1_winner = predict_match_winner(semi_finalists['Group 1'][0], semi_finalists['Group 2'][1], df)
    semi_final_2_winner = predict_match_winner(semi_finalists['Group 2'][0], semi_finalists['Group 1'][1], df)
    print(f"Semi-Final 1: {semi_finalists['Group 1'][0]} vs {semi_finalists['Group 2'][1]} -> Winner: {semi_final_1_winner}")
    print(f"Semi-Final 2: {semi_finalists['Group 2'][0]} vs {semi_finalists['Group 1'][1]} -> Winner: {semi_final_2_winner}")
    return semi_final_1_winner, semi_final_2_winner

semi_final_1_winner, semi_final_2_winner = simulate_semi_finals(semi_finalists, df)

def simulate_final(semi_final_1_winner, semi_final_2_winner, df):
    champion = predict_match_winner(semi_final_1_winner, semi_final_2_winner, df)
    print(f"\nFinal: {semi_final_1_winner} vs {semi_final_2_winner} -> Winner: {champion}")
    return champion

champion = simulate_final(semi_final_1_winner, semi_final_2_winner, df)

print("\nChampion:")
print(f"{champion}")


Groups:
Group A: Canada, India, Ireland, Pakistan, United States
Group B: Australia, England, Namibia, Oman, Scotland
Group C: Afghanistan, New Zealand, Papua New Guinea, Uganda, West Indies
Group D: Bangladesh, Netherlands, South Africa, Sri Lanka, Nepal

Group A:
Match 1: Canada vs India -> Winner: India
Match 2: Canada vs Ireland -> Winner: Ireland
Match 3: Canada vs Pakistan -> Winner: Pakistan
Match 4: Canada vs United States -> Winner: Canada
Match 5: India vs Ireland -> Winner: India
Match 6: India vs Pakistan -> Winner: India
Match 7: India vs United States -> Winner: India
Match 8: Ireland vs Pakistan -> Winner: Pakistan
Match 9: Ireland vs United States -> Winner: Ireland
Match 10: Pakistan vs United States -> Winner: Pakistan

Group B:
Match 11: Australia vs England -> Winner: Australia
Match 12: Australia vs Namibia -> Winner: Australia
Match 13: Australia vs Oman -> Winner: Australia
Match 14: Australia vs Scotland -> Winner: Australia
Match 15: England vs Namibia -> Winn